# Théorie de l’explosion thermique

## Historique

La théorie de l'explosion thermique trouve son origine dans les travaux de Nikolay N. Semenov, qui même s'il est chimiste, a travaillé aux interfaces des disciplines (mathématique, physique,...) dans la pure tradition russe de l'époque.


<img src="images/semenov.jpg" alt="N.N. Semenov" style="width: 300px;"/>
  
>**Nikolay Nikolayevich Semyonov** (Семёнов) : 
>In the laboratories of the professor Boris Veinberg there worked  Nikolay Semenov, the future Nobel Prize winner for physical chemistry (1956). He was born in Saratov (1896), graduated from the Saratov Realschule and joined the Math Branch of the Math and Physics Department at Petrograd Imperial University. After graduation, he stayed at the university as a professorial fellow.

Ses travaux couvrent principalement l'étude des mécanismes de transformation chimique et en particulier une étude exhaustive de la théorie des réactions en chaine, avec des applications en particulier aux processus de combustion.


<img src="images/stele_semenov.jpg" alt="N.N. Semenov" style="width: 300px;"/>
 
>**Nikolay Nikolayevich Semyonov** (1896-1996): 
>In the spring of 1918 Semenov went on vacation to his parents in Samara, where he was called to arms to the Kolchak White People’s Army. After doing his service for 3 weeks, he deserted and moved to Tomsk, which was the nearest city with a university. Until 1920 Nikolay Semenov did postgraduate studies in the Tomsk Institute of Technology, and then he returned to Petrograd.
In 1931 he established the Institute of Chemical Physics of Russian Academy of Science (ICP RAS) and stayed as its irreplaceable director until the end of his days. Together with the Soviet physicist Pyotr Leonidovich Kapitsa, Nikolay Semenov founded the Moscow Institute of Physics and Technology in 1946.
Nikolay Semenov was the only Soviet physicist and chemist who was awarded the Nobel Prize.

Le second nom associé à la théorie de l'explosion thermique est celui de David A. Frank-Kamenetskii. Invité par Semenov à qui il avait envoyé une lettre sur la thermodynamique chimique, il a travaillé pendant sept and de 1934 à 1941 à l'Institut de Chimie Physique de l'académie des sciences de Russie fondé par Semenov, où il a travaillé sur les problèmes d'explosion thermique et rencontré une autre figure de la communauté Russe de la combustion : Yakov Borisovich Zel'dovich, avec qui il deviendra très ami.


<img src="images/kamenetskii.jpg" alt="D.A. Frank-Kamenetskii" style="width: 200px;"/>
  
>**David Albertovich Frank-Kamenetskii** (1910-1970): 
>He was a Soviet theoretical physicist and chemist, professor and doctor of physical, chemical and mathemetical sciences. He developed the thermal explosion theory, worked on plasma physics problems and in astrophysics.

In [1]:
import numpy as np
from scipy.integrate import solve_ivp

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column

from ipywidgets import interact, FloatSlider

output_notebook(hide_banner=True)

## Modèle simplifié 1 - explosion adiabatique


**Non linéarité de $\displaystyle\Lambda(T) = B e^{-\beta T_0/T} (T_b - T)$**

In [2]:
T0 = 400.
Tb = 3000.
T = np.linspace(T0,Tb,1000)

beta = 30.

Lambda = np.exp(-(beta*T0)/T) * (Tb-T)
LambdaoverLambdamax = Lambda / np.max(Lambda)

#print()

output_notebook(hide_banner=True)
p = figure(x_range=(T0, Tb+10), plot_width=980, plot_height=500, title="Evolution de \u039B(T)")
plt = p.line(T, LambdaoverLambdamax, line_width=2)
show(p, notebook_handle=True)

@interact(beta=FloatSlider(min=30 ,max=300,step=10,value=30))
def update(beta) :
    Lambda = np.exp(-(beta*T0)/T) * (Tb-T)
    #print(-(beta*T0))

    LambdaoverLambdamax = Lambda / np.max(Lambda)
    print(np.max(Lambda))
    plt.data_source.data = dict(x=T, y=LambdaoverLambdamax)
    push_notebook()

interactive(children=(FloatSlider(value=30.0, description='beta', max=300.0, min=30.0, step=10.0), Output()), …

**Evolution temporelle  de $\widetilde{\theta}$ solution de $\, d_{\tau} \widetilde{\theta} = e^{\widetilde{\theta}}$**

In [3]:
from mylib.model import first_model

fm = first_model()
fcn = fm.fcn

tini = 0.0
tend = 1.0

yini = (0.0,)

theta_max = 10

sol = solve_ivp(fcn, (tini, tend), yini, method='RK45', rtol=1.e-12, atol=1.e-12)

fig = figure(plot_height=300, plot_width=900, x_range=(tini, tend+0.05), y_range=(0, theta_max), 
             title = "Evolution temporelle de \u03B8(t)")
fig.line(sol.t, sol.y[0], line_width=3)
show(fig)

## Modèle 2 : explosion avec prise en compte des pertes thermiques

**Graphes de  $e^\widetilde{\theta}$ et de $q^- (\widetilde{\theta})= \alpha_0 \, \widetilde{\theta}$**

In [18]:
alpha = 1.

theta = np.linspace(0, 2.5,1000)

exp = np.exp(theta)
q = alpha*theta

output_notebook(hide_banner=True)
p = figure(x_range=(0, 2.5), y_range=(0, 10.), plot_width=980, plot_height=500)
plt_exp = p.line(theta, exp, line_width=2, legend = "exp(\u03B8)")
plt_q = p.line(theta, q, color="Crimson", line_width=2, legend = "q")
p.legend.location = "top_left"
show(p, notebook_handle=True)

e = np.exp(1)
eover10 = np.exp(1) / 10

@interact(alpha=FloatSlider(min=e-5*eover10 ,max=e+5*eover10, step=eover10, value=e))
def update(alpha) :
    q = alpha*theta
    plt_q.data_source.data = dict(x=theta, y=q)
    push_notebook()

interactive(children=(FloatSlider(value=2.718281828459045, description='alpha', max=4.077422742688567, min=1.3…

**Evolution temporelle  de $\widetilde{\theta}$ solution de $d_{\tau} \widetilde{\theta} = e^\widetilde{\theta}  - \alpha_0 \, \widetilde{\theta} $**

In [14]:
from mylib.model import second_model

alpha = 3.0

sm = second_model(alpha)
fcn = sm.fcn

tini = 0.0
tend = 30.0

yini = (0.0,)

sol = solve_ivp(fcn, (tini, tend), yini, method='RK45', atol=1.e-12, rtol=1.e-12)

fig = figure(plot_height=300, plot_width=900, x_range=(tini, tend), title = "Evolution temporelle de \u03B8(t)")
plt = fig.line(sol.t, sol.y[0], line_width=2)
show(fig, notebook_handle=True)

@interact(alpha = FloatSlider(min=0.0, max=5.0, step=0.1, value=3.0, continuous_update=False))
def update(alpha) :
    sm = second_model(alpha)
    fcn = sm.fcn
 
    sol = solve_ivp(fcn, (tini, tend), yini, method='RK45', atol=1.e-12, rtol=1.e-12)
    
    plt.data_source.data = dict(x=sol.t, y=sol.y[0])
    
    push_notebook()

interactive(children=(FloatSlider(value=3.0, continuous_update=False, description='alpha', max=5.0), Output())…

####  Modèle de l'explosion avec prise en compte des pertes thermiques et consommation de combustible
$$
\left\{\begin{aligned}
d_{\tau} \widetilde{\theta} & = \exp\Bigg(\frac{\widetilde{\theta}}{1 + \widetilde{\theta} \, / \, \beta}\Bigg) \widetilde{Y} - \alpha_0 \, \widetilde{\theta} \\
d_{\tau} \widetilde{Y} & = -\frac{1}{\overline T_r} \exp\Bigg(\frac{\widetilde{\theta}}{1 + \widetilde{\theta} \, / \, \beta}\Bigg)  \widetilde{Y}
\end{aligned}\right.
$$

In [24]:
from mylib.model import second_bis_model

alpha = 0.0
Tr = 200.0
beta = 30.

sbm = second_bis_model(alpha, Tr, beta)
fcn = sbm.fcn

tini = 0.0
tend = 10.0

yini = (0.0, 1.0)

sol = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)

fig_theta = figure(plot_height=300, plot_width=900, x_range=(tini, tend), y_range=(0, Tr), 
                   title = "Evolution temporelle de \u03B8(t)")
plt_theta = fig_theta.line(sol.t, sol.y[0], line_width=2)
fig_Y = figure(plot_height=300, plot_width=900, x_range=(tini, tend), title = "Evolution temporelle de Y(t)")
plt_Y = fig_Y.line(sol.t, sol.y[1], line_width=2)

show(column(fig_theta, fig_Y), notebook_handle=True)

@interact(alpha = FloatSlider(min=0.0, max=3.0, step=0.1, value=0.0, continuous_update=False))
def update(alpha) :
    sbm = second_bis_model(alpha, Tr, beta)
    fcn = sbm.fcn

    sol = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)
    
    plt_theta.data_source.data = dict(x=sol.t, y=sol.y[0])
    plt_Y.data_source.data = dict(x=sol.t, y=sol.y[1])
    
    push_notebook()

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='alpha', max=3.0), Output())…

**Par linéarisation de Frank-Kamenetski, le modèle précédent prend la forme suivante :**

$$
\left\{\begin{aligned}
d_{\tau} \widetilde{\theta} & = e^{\widetilde{\theta}} \widetilde{Y} - \alpha_0 \, \widetilde{\theta} \\
d_{\tau} \widetilde{Y} & = -\frac{1}{\overline T_r} e^{\widetilde{\theta}} \widetilde{Y}
\end{aligned}\right.
$$

In [19]:
from mylib.model import second_bis_fk_model

alpha = 0.0
Tr = 30.0

sbfkm = second_bis_fk_model(alpha, Tr)
fcn = sbfkm.fcn

tini = 0.0
tend = 10.0

yini = (1.4, 1.0)

sol = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)

fig_theta = figure(plot_height=300, plot_width=900, x_range=(tini, tend), y_range=(0, Tr), 
                   title = "Evolution temporelle de \u03B8(t)")
plt_theta = fig_theta.line(sol.t, sol.y[0], line_width=2)
fig_Y = figure(plot_height=300, plot_width=900, x_range=(tini, tend), title = "Evolution temporelle de Y(t)")
plt_Y = fig_Y.line(sol.t, sol.y[1], line_width=2)

show(column(fig_theta, fig_Y), notebook_handle=True)

@interact(alpha = FloatSlider(min=0.0, max=3.0, step=0.1, value=1.0, continuous_update=False))
def update(alpha) :
    sbfkm = second_bis_fk_model(alpha, Tr)
    fcn = sbfkm.fcn

    sol = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)
    
    plt_theta.data_source.data = dict(x=sol.t, y=sol.y[0])
    plt_Y.data_source.data = dict(x=sol.t, y=sol.y[1])
    
    push_notebook()

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='alpha', max=3.0), Output())…

In [23]:
from mylib.model import second_bis_fk_model_2 as second_bis_fk_model

alpha = 0.0
Tr = 30.0

sbfkm = second_bis_fk_model(alpha, Tr)
fcn = sbfkm.fcn

tini = 0.0
tend = 10.0

yini = (1.6, 0.4)

sol = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)

fig_theta = figure(plot_height=300, plot_width=900, x_range=(tini, tend), 
                   title = "Evolution temporelle de \u03B8(t)") # y_range=(0, 1)
plt_theta = fig_theta.line(sol.t, sol.y[0], line_width=2)
fig_Y = figure(plot_height=300, plot_width=900, x_range=(tini, tend), title = "Evolution temporelle de Y(t)")
plt_Y = fig_Y.line(sol.t, sol.y[1], line_width=2)

show(column(fig_theta, fig_Y), notebook_handle=True)

@interact(alpha = FloatSlider(min=0.0, max=3.0, step=0.1, value=1.0, continuous_update=False))
def update(alpha) :
    sbfkm = second_bis_fk_model(alpha, Tr)
    fcn = sbfkm.fcn

    sol = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)
    
    plt_theta.data_source.data = dict(x=sol.t, y=sol.y[0])
    plt_Y.data_source.data = dict(x=sol.t, y=sol.y[1])
    
    push_notebook()

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='alpha', max=3.0), Output())…

## Modèle simplifié 3 - explosion avec prise en compte de la convection

$$
\left\{\begin{aligned}
d_{\tau} \widetilde{\theta} & = e^{\widetilde{\theta}} - \alpha_0 \, (1 + \mu \, \psi^2) \, \widetilde{\theta} \\
d_{\tau} \psi & = -a \, \psi (\psi^2 + \widetilde{\theta_c} - \widetilde{\theta})
\end{aligned}\right.
$$

In [3]:
from mylib.model import third_model


alpha0=2.7
mu=0.7
a=4.3
thetac=1.035

tm = third_model(alpha0, mu, a, thetac)
fcn = tm.fcn

tini = 0. 
tend = 3000.

yini = (1.0, 1.1)

sol = solve_ivp(fcn, (tini, tend), yini, method='RK45', atol=1.e-8, rtol=1.e-8)

fig_theta = figure(plot_height=300, plot_width=900, x_range=(tini, tend), title = "Evolution temporelle de \u03B8(t)")
plt_theta = fig_theta.line(sol.t, sol.y[0], line_width=2)
fig_phase = figure(plot_height=300, plot_width=900, title = "Plan de phase (\u03B8, \u03C8)")
plt_phase = fig_phase.line(sol.y[0], sol.y[1])

show(column(fig_theta, fig_phase), notebook_handle=True)

@interact(thetac=FloatSlider(min=1.00,max=1.065,step=0.0025,value=1.03,readout_format='.3f', continuous_update=False))
def update(thetac) :
    tm = third_model(alpha0, mu, a, thetac)
    fcn = tm.fcn

    sol = solve_ivp(fcn, (tini, tend), yini, method='RK45', atol=1.e-8, rtol=1.e-8)
    
    plt_theta.data_source.data = dict(x=sol.t, y=sol.y[0])
    plt_phase.data_source.data = dict(x=sol.y[0], y=sol.y[1])
    
    push_notebook()

interactive(children=(FloatSlider(value=1.03, continuous_update=False, description='thetac', max=1.065, min=1.…